# Poços Activity Analysis

This notebooks perform a basic EDA on a 230km activity near Poços de Caldas / MG, and compares some of the real measurements against the model predictions

In [1]:
%load_ext autotime

import sys
sys.path.append("../..")

time: 993 µs (started: 2021-05-07 22:16:20 -03:00)


In [2]:
import pandas as pd
import plotly.express as px


time: 2.01 s (started: 2021-05-07 22:16:20 -03:00)


In [3]:
import json
with open('../../data/pocos.json', 'r') as fid:
    data = json.load(fid)

records = [el['data'] for el in data if el['type'] == 'record']
df = (pd.DataFrame(records)
        .assign(timestamp=lambda df: pd.to_datetime(df.timestamp))
        .assign(seconds_passed=lambda df: (df.timestamp - df.timestamp[0]).dt.total_seconds())
        )


time: 2.96 s (started: 2021-05-07 22:16:22 -03:00)


In [4]:
fig_df = df.query('seconds_passed < 7000')
px.line(fig_df,
        x='seconds_passed',
        y='speed')

time: 1.35 s (started: 2021-05-07 22:16:25 -03:00)


In [5]:
from scipy.interpolate import interp1d
elevation_interpolator = interp1d((df.distance - df.distance[0]) * 1000, df.altitude)


time: 133 ms (started: 2021-05-07 22:16:27 -03:00)


In [6]:
from cadCAD_tools import easy_run
from model import initial_state, params, timestep_block
params.update(elevation_interpolator=[elevation_interpolator],
              dt=[0.5])

results = easy_run(initial_state,
                   params,
                   timestep_block,
                   10000,
                   1,
                   assign_params=False)

Initializing configurations: 100%|██████████| 1/1 [00:00<00:00, 50.26it/s]
                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (10000, 6, 1, 6)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded

Flattening results:   0%|          | 1/40001 [00:00<19:00, 35.08it/s]
Total execution time: 6.64s
time: 7.21 s (started: 2021-05-07 22:16:27 -03:00)


In [7]:
df_1 = (results.assign(speed=lambda df: df.speed * 3.6).assign(model=True).assign(distance=lambda df: df.position / 1000))
df_2 = df.assign(model=False)
df_3 = pd.concat([df_1, df_2]).query(f"distance < {df_1.distance.max()} & distance > 0.5")


px.line(df_3,
        x='distance',
        y='speed',
        color='model')

time: 820 ms (started: 2021-05-07 22:16:34 -03:00)


In [8]:
df_3.query('model == True').seconds_passed.max() / df_3.query('model == False').seconds_passed.max()

0.9469596514491381

time: 61.7 ms (started: 2021-05-07 22:16:35 -03:00)


5346.0

time: 20.3 ms (started: 2021-05-07 22:12:53 -03:00)
